In [3]:
import numpy as np

### Task 0: Initialize Gaussian Process

In [4]:
"""Task 0 - 2"""


class GaussianProcess:
    """Represents a noiselsess 1D Gaussian process"""

    def __init__(self, X_init, Y_init, l=1, sigma_f=1):
        """Constructor
        Args:
                X_init: np.ndarray of shape (t, 1) representing the inputs
                    already sampled with the black-box function
                Y_init: np.ndarray of shape (t, 1) representing the outputs
                    of the black-box function for each input in X_init
                t: number of initial samples
                l: length parameter for the kernel
                sigma_f: standard deviation given to the output of the
                    black-box function"""
        self.X = X_init
        self.Y = Y_init
        self.l = l
        self.sigma_f = sigma_f
        self.K = self.kernel(X_init, X_init)

    def kernel(self, X1, X2):
        """Calclulates the covariance kernel matrix between two matrices
        Args:
            X1: np.ndarray of shape (m, 1)
            X2: np.ndarray of shape (n, 1)
            uses Radial Basis Function (RBF)
        Returns: covariance kernel matrix as a np.ndarray of shape (m, n)"""

        sqdist = np.sum(X1**2, 1).reshape(-1, 1) +\
            np.sum(X2**2, 1) - 2 * np.dot(X1, X2.T)
        return self.sigma_f**2 * np.exp(-0.5 / self.l**2 * sqdist)

    def predict(self, X_s):
        """Predicts the mean and standard deviation of points in a
        Gaussian process
        Args:
            X_s: np.ndarray of shape (s, 1) containing all of the points
        Returns: mu, sigma
            mu: np.ndarray of shape (s,) containing the mean for each point in X_s
            sigma: np.ndarray of shape (s,) containing the variance for each    point in X_s"""
        K = self.K
        K_s = self.kernel(self.X, X_s)
        K_ss = self.kernel(X_s, X_s)
        K_inv = np.linalg.inv(K)

        mu_s = K_s.T.dot(K_inv).dot(self.Y)
        mu_s = mu_s.reshape(-1)

        cov_s = K_ss - K_s.T.dot(K_inv).dot(K_s)
        var_s = np.diag(cov_s)

        return mu_s, var_s

    def update(self, X_new, Y_new):
        """Updates a Gaussian Process - attributes X, Y, and K
        Args:
            X_new: np.ndarray of shape (1,) that represents the new sample
            point
            Y_new: np.ndarray of sahpe (1,) that represents the new sample
            function value"""

        self.X = np.append(self.X, X_new)
        self.X = self.X[:, np.newaxis]
        self.Y = np.append(self.Y, Y_new)
        self.Y = self.Y[:, np.newaxis]
        self.K = self.kernel(self.X, self.X)
        

### Tasks 3 - 5

In [32]:
class BayesianOptimization:
    """Performs Bayesian optimization on a noiseless 1D Gaussian process
    """

    def __init__(self, f, X_init, Y_init, bounds, ac_samples, l=1, sigma_f=1,
                 xsi=0.01, minimize=True):
        """Constructor to set instance attributes
        Args:
            f: black-box function to be optimized
            X_init: np.ndarray of shape (t, 1) representing the inputs
                already sampled with the black-box function
            Y_init: np.ndarray of shape (t, 1) representing the outputs
                t: number of initial samples
            bounds: tuple of (min, max) representing the bounds of the space 
                to look for the optimal point
            ac_samples: number of samples that should be analyzed during
                acquisition
            l: length parameter for the kernel
            sigma_f: standard deviation given to the output of the
                black-box function
            xsi: exploration-exploitation factor for acquisition
            minimize: bool determining whether optimization should be
                performed for minimization (True) or maximization (False)
        """

        self.f = f
        self.gp = GaussianProcess(X_init, Y_init, l, sigma_f)
        self.X_s = np.linspace(bounds[0], bounds[1], ac_samples).reshape(-1, 1)
        self.xsi = xsi
        self.minimize = minimize

    def acquisition(self):
        """Calculated the next best sample location with Expected
        Improvement acquisition function
        Returns:
            X_next: np.ndarray of shape (1,) representing the next best
                sample point
            EI: np.ndarray of shape (ac_samples,) containing the expected
                improvement of each potential sample"""

        from scipy.stats import norm
        # getting mu and sigma from current sample points
        mu, _ = self.gp.predict(self.gp.X)
        # getting mu and sigma from potential sample points
        mu_sample, sigma_sample = self.gp.predict(self.X_s)

        # Defining if minimization or maximization of acquisition function
        if self.minimize is True:
            mu_sample_opt = np.min(mu)
        else:
            mu_sample_opt = np.max(mu)

        # Expected Improvement Calculation
        with np.errstate(divide='warn'):
            imp = mu_sample_opt - mu_sample - self.xsi
            Z = imp / sigma_sample
            ei = imp * norm.cdf(Z) + sigma_sample * norm.pdf(Z)
            ei[sigma_sample == 0.0] = 0.0

        X_next = self.X_s[np.argmax(ei)]

        return X_next, np.array(ei)

    def optimize(self, iterations=100):
        """Optimizes the black-box function - stops early if proposed point is
        one that has already been sampled
        Args:
            iterations: maximum number of iterations to perform
        Returns:
            X_opt: np.ndarray of shape (1,) representing the optimal point
            Y_opt: np.ndarray of shape (1,) representing the optimal point
                value"""
        for _ in range(iterations):
            X_next, _ = self.acquisition()
            if X_next in self.gp.X:
                break
            Y_next = self.f(X_next)
            self.gp.update(X_next, Y_next)

        idx = np.argmin(self.gp.Y)
        return self.gp.X[idx], np.array(self.gp.Y[idx])


In [33]:
def f(x):
    """our 'black box' function"""
    return np.sin(5*x) + 2*np.sin(-2*x)

if __name__ == '__main__':
    np.random.seed(0)
    X_init = np.random.uniform(-np.pi, 2*np.pi, (2, 1))
    Y_init = f(X_init)

    bo = BayesianOptimization(f, X_init, Y_init, (-np.pi, 2*np.pi), 50, l=0.6, sigma_f=2)
    X_opt, Y_opt = bo.optimize(50)
    print('Optimal X:', X_opt)
    print('Optimal Y:', Y_opt)
    print('All sample inputs:', bo.gp.X)

Optimal X: [0.8975979]
Optimal Y: [-2.92478374]
All sample inputs: [[ 2.03085276]
 [ 3.59890832]
 [ 4.55210364]
 [ 5.89850049]
 [-2.94925025]
 [-1.7951958 ]
 [-0.44879895]
 [ 0.70525549]
 [ 3.01336438]
 [ 3.97507642]
 [ 1.28228272]
 [ 5.12913086]
 [-2.37222302]
 [ 6.28318531]
 [ 0.32057068]
 [-1.21816858]
 [ 0.8975979 ]
 [-3.14159265]
 [ 2.43633716]]


/tmp/ipykernel_474/3322794705.py:56: RuntimeWarning: divide by zero encountered in divide
  Z = imp / sigma_sample
